## A simple implementation of TD0 and TD_lambda with OpenAI Gym FrozenLakeNotSlippery-v0

This implement the basic Q value(state-action value) table lookup using SARSA update and Q learning to walk a maze.

SARSA update:
$Q(state_t,action_t) \leftarrow Q(state_t,action_t) + \alpha[reward + \gamma * Q(state_{t+1},action_{t+1}) - Q(state_t,action_t)]$

Q learning:
$Q(state_t,action_t) \leftarrow Q(state_t,action_t) + \alpha[reward + \gamma * max(Q(state_{t+1},action)) - Q(state_t,action_t)]$

In [4]:
import gym, random
import time
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery4x4-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
)
register(
    id='FrozenLakeNotSlippery8x8-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '8x8', 'is_slippery': False},
)

FrozenLake is a grid world with some randomness, as the agent and slip and rumble into a different cell it intended to walk to.
The FrozenLakeNotSlippery-v0 environment is same as FrozenLake except that it remove the randomness.  
S - is where the agent start  
F - is safe cell  
H - is hole, when agent hit a hole, episode end with 0 reward  
G - is goal state, 1.0 reward  

In [5]:
env = gym.make('FrozenLakeNotSlippery4x4-v0')
env.render()


SFFF
FHFH
FFFH
HFFG


## TD0 with Q value table look up

In [36]:
class TD0(object):
  
  def __init__(self):
    self.Q = {}
    self.iteration = 0
    self._gamma = 0.5
    self._alpha = 0.02
    self._epsilon = 0.4

    # init Q, FrozenLake environment contain a grid with 64 cells, 
    # and 4 actions to choose from. setting all (state, action) pair to 0
    for i in range(64):
      self.Q[i] = [0.0] * 4

  def alpha(self):
    decay = 1 / ( 1 + self.iteration / 5000)
    return self._alpha * decay
  
  def epsilon(self):
    decay = 1 / ( 1 + self.iteration / 5000)
    return self._epsilon * decay

  def update_sarsa(self, s0, a0, r, s1, a1):
    # on policy SARAS update
    # using bellman equation, q value target is r + discount * Q(next_state, next_action)
    # delta = target - current state action estimation
    if a1 != None:
      delta = r + self._gamma * self.Q[s1][a1] - self.Q[s0][a0]
    else:
      delta = r - self.Q[s0][a0]
    self.Q[s0][a0] += self.alpha() * delta
    
  def pick_action(self, state):
    if random.random() < self.epsilon():
      return random.randrange(0,4,1)
    else:
      return self.Q[state].index(max(self.Q[state]))
    
  def show_policy(self):
    # greedy policy
    action_name_map = ['left','down','right','up','none']
    for i in range(4):
      policy = ""
      for j in range(4):
        state = i*4+j
        action_index = self.Q[state].index(max(self.Q[state]))
        if self.Q[state] == [0.0]*4:
          action_index = 4
        policy += "%2d: %-10s" %(state, action_name_map[action_index])
      print(policy)
    
  def train(self, env, episode, use_step_cost = False):
    for i in range(episode):
      self.iteration += 1
      s0 = env.reset()
      a0 = self.pick_action(s0)
      episode_ended = False
      while not episode_ended:
        (s1, reward, episode_ended, info) = env.step(a0)
        # set use_step_cost = True, the agent should learn the shortest path
        if use_step_cost and reward <= 0.0:
            reward = -0.1
            if episode_ended:
              # walk into hole
              reward = -1.0
        if not episode_ended:
          a1 = self.pick_action(s1)
        else:
          a1 = None
        self.update_sarsa(s0,a0,reward,s1,a1)
        s0 = s1
        a0 = a1

In [39]:
env = gym.make('FrozenLakeNotSlippery4x4-v0')
agent = TD0()

In [40]:
for i in range(3):
  agent.train(env, 5000)
  agent.show_policy()
  print ("=" * 50)

 0: none       1: none       2: none       3: none      
 4: none       5: none       6: none       7: none      
 8: none       9: none      10: none      11: none      
12: none      13: none      14: right     15: none      
 0: down       1: left       2: left       3: none      
 4: down       5: none       6: up         7: none      
 8: right      9: down      10: down      11: none      
12: none      13: right     14: right     15: none      
 0: down       1: left       2: left       3: none      
 4: down       5: none       6: up         7: none      
 8: right      9: down      10: down      11: none      
12: none      13: right     14: right     15: none      


Remark: The Agent may not learn the shortest path as the cost / reward for taking a move is 0. So walking a longer route is having the same reward as walking a shorter path. To make the agent learn the shortest path, set reward to a small negative value (e.g. -0.1) and a bigger cost falling into a hole can make the agent prefer shorter path.

## TD Lambda with Q value table lookup using SARAS update and Q update

In [44]:
class TD_lambda(object):
  
  def __init__(self, grid_w, grid_h):
    # init Q, FrozenLake environment contain a grid with 16 cells, 
    # and 4 actions to choose from 
    self.Q = {}
    self.iteration = 0
    self._gamma = 0.9
    self._lambda = 0.5
    self._alpha = 0.02
    self._epsilon = 0.4

    self.grid_w = grid_w
    self.grid_h = grid_h
    for i in range(grid_w * grid_h):
      self.Q[i] = [0.0] * 4
      
  def alpha(self):
    # maybe a bit over kill to wrap alpha in a function,
    # but this make it easy if want to do alpha decay 
    decay = 1 / ( 1 + self.iteration / 5000)
    return self._alpha * decay
  
  def epsilon(self):
    # same as alpha, wrap in a function make it easier to do epsilon decay if needed
    decay = 1 / ( 1 + self.iteration / 5000)
    return self._epsilon * decay

  def update_eligibility(self):
    for key in self.eligibility:
      self.eligibility[key] *= self._gamma * self._lambda

  def update_sarsa(self, s0, a0, r, s1, a1):
    if a1 != None:
      delta = r + self._gamma * self.Q[s1][a1] - self.Q[s0][a0]
    else:
      delta = r - self.Q[s0][a0]
    for (s,a) in self.eligibility:
      self.Q[s][a] += self.alpha() * delta * self.eligibility[s,a]

  def update_Q(self, s0, a0, r, s1, a1):
    if a1 != None:
      delta = r + self._gamma * max(self.Q[s1]) - self.Q[s0][a0]
    else:
      delta = r - self.Q[s0][a0]
    for (s,a) in self.eligibility:
      self.Q[s][a] += self.alpha() * delta * self.eligibility[s,a]

  def pick_action(self, state):
    if random.random() < self.epsilon():
      return random.randrange(0,4,1)
    else:
      return self.Q[state].index(max(self.Q[state]))

  def show_policy(self):
    # greedy policy
    action_name_map = ['left','down','right','up','none']
    for i in range(self.grid_h):
      policy = ""
      for j in range(self.grid_w):
        state = i * self.grid_w + j
        action_index = self.Q[state].index(max(self.Q[state]))
        if self.Q[state] == [0]*4:
          action_index = 4
        policy += "%2d: %-10s" %(state, action_name_map[action_index])
      print(policy)

  def train(self, env, episode, update='sarsa', use_step_cost = False):
    for i in range(episode):
      self.iteration += 1
      s0 = env.reset()
      a0 = self.pick_action(s0)
      self.eligibility = {}
      episode_ended = False
      while not episode_ended:
        (s1, reward, episode_ended, info) = env.step(a0)
        # agent should learn the shortest path with step cost (with enough iteration)
        # step cost helps to converge faster in grid world too.
        if use_step_cost and reward <= 0.0:
            reward = -0.1
            if episode_ended:
              reward = -1

        if (s0,a0) in self.eligibility:
          self.eligibility[s0,a0] += 1
        else:
          self.eligibility[s0,a0] = 1
        if not episode_ended:
          a1 = self.pick_action(s1)
        else:
          a1 = None
        if update == 'sarsa':
          self.update_sarsa(s0,a0,reward,s1,a1)
        elif update == 'Q':
          self.update_Q(s0,a0,reward,s1,a1)
        self.update_eligibility()
        s0 = s1
        a0 = a1

In [45]:
env = gym.make('FrozenLakeNotSlippery4x4-v0')
agent_lambda = TD_lambda(4,4)

In [46]:
for i in range(3):
  # set update to sarsa to use on policy SARSA update
  agent_lambda.train(env, 5000, update='sarsa')
  agent_lambda.show_policy()
  print ("=" * 50)

 0: none       1: none       2: none       3: none      
 4: none       5: none       6: none       7: none      
 8: none       9: none      10: none      11: none      
12: none      13: none      14: none      15: none      
 0: down       1: left       2: left       3: none      
 4: down       5: none       6: none       7: none      
 8: right      9: down      10: left      11: none      
12: none      13: right     14: right     15: none      
 0: down       1: left       2: left       3: none      
 4: down       5: none       6: none       7: none      
 8: right      9: down      10: left      11: none      
12: none      13: right     14: right     15: none      


In [19]:
env = gym.make('FrozenLakeNotSlippery4x4-v0')
agent_lambda = TD_lambda(4,4)

In [20]:
for i in range(3):
  # set update to sarsa to use Q update 
  agent_lambda.train(env, 5000, update='Q')
  agent_lambda.show_policy()
  print ("=" * 50)

 0: none       1: none       2: none       3: none      
 4: none       5: none       6: none       7: none      
 8: none       9: none      10: none      11: none      
12: none      13: none      14: none      15: none      
 0: down       1: left       2: left       3: none      
 4: down       5: none       6: up         7: none      
 8: right      9: down      10: down      11: none      
12: none      13: right     14: right     15: none      
 0: down       1: left       2: left       3: left      
 4: down       5: none       6: up         7: none      
 8: right      9: right     10: down      11: none      
12: none      13: right     14: right     15: none      


In [21]:
env = gym.make('FrozenLakeNotSlippery8x8-v0')
env.render()
agent_lambda = TD_lambda(8,8)


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [47]:
for i in range(8):
  agent_lambda.train(env, 20000, update='Q', use_step_cost=True)
agent_lambda.show_policy()
print ("=" * 110)

 0: down       1: left       2: left       3: none      
 4: down       5: none       6: down       7: none      
 8: right      9: down      10: left      11: none      
12: none      13: right     14: right     15: none      


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import random
from itertools import combinations

In [36]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor

In [49]:
class NN(nn.Module):
  def __init__(self, in_size, out_size):
    super(NN, self).__init__()
    self.in_size = in_size
    self.out_size = out_size
    self.l1_linear = nn.Linear(in_size, 4, bias=False)
#     self.l2_linear = nn.Linear(4,out_size, bias=False)
  
  def forward(self, x):
    out = self.l1_linear(x)
#     out = F.sigmoid(self.l1_linear(x))
#     out = F.sigmoid(self.l2_linear(out))
    return out

class TransitionModel():
  state = {}
  total_count = 0
  def __init__(self):
    pass
  
  def clear(self):
    self.state = {}
    self.total_count = 0
  
  def add_transition(self, s0, a0, r, s1):
    if (s0,a0) not in self.state:
      self.state[s0,a0] = {
        't':{},
        'c':0
      }
    transition = self.state[s0,a0]['t']
    if (r, s1) not in transition:
      transition[r, s1] = 0
    transition[r, s1] += 1
    self.state[s0, a0]['c'] += 1
    self.total_count += 1

  def get_transition(self, s0, a0):
    if (s0,a0) not in self.state:
      return None
    threshold = random.random()
    count = 0.0
    transition = self.state[s0,a0]['t']
    total_count = self.state[s0,a0]['c']
    for key in transition:
      count += transition[key] / total_count * 1.0
      if count > threshold:
        return key

  def get_random_transition(self, sample_count=1):
    transitions = []
    start_states = list(self.state.keys())
    start_states_thresholds = []
    tmp = 0.0
    for i in range(len(start_states)):
      tmp += self.state[start_states[i]]['c'] * 1.0 / self.total_count
      start_states_thresholds.append(tmp)

    for i in range(sample_count):
      threshold = random.random()
      idx = 0
      for j in range(len(start_states)):
        if start_states_thresholds[j] > threshold:
          idx = j
          break
      (s0, a0) = start_states[idx]
      (r, s1) = self.get_transition(s0, a0)
      transitions.append((s0,a0,r,s1))
    return transitions

class TransitionHistory():
  transitions = []
  def __init__(self):
    pass
  
  def clear(self):
    self.transitions = []
  
  def add_transition(self, s0, a0, r, s1):
    self.transitions.append((s0,a0,r,s1))
  
  def get_sample_transition(self, batch_size):
    return random.sample(self.transitions, batch_size)

class q_approx():
  _gamma = 0.8
  _lambda = 0.5
  _epsilon = 0.2
  transition_history = TransitionHistory()

  def __init__(self, size):
    self.Q = NN(size, 4)
    if use_cuda:
      self.Q.cuda()
    self.size = size
    self.optimizer = torch.optim.Adagrad(self.Q.parameters(),weight_decay=1e-5)
#     self.optimizer = torch.optim.SGD(self.Q.parameters(),lr=2.0)
    self.loss_fn = nn.MSELoss()

  def index_to_onehot(self, index, total=16):
    onehot = [0] * total
    onehot[index] = 1
    return onehot
  
  def epsilon(self):
    return self._epsilon

  def pick_action(self, state):
    if random.random() < self.epsilon():
      return random.randrange(0,4,1)
    else:
      self.predict(state)
      onehot = self.index_to_onehot(state)
      s = Variable(FloatTensor([onehot]))
      action_value = self.Q(s).data.tolist()
      return action_value.index(max(action_value))

  def predict(self, state):
    onehot = self.index_to_onehot(state)
    s = Variable(FloatTensor([onehot]))
    action_value = self.Q(s).data.tolist()[0]
    return action_value

  def update_Q(self, batch):
#     print(batch)
    (state, action, reward, next_state) = tuple(zip(*batch))
    non_final_mask = torch.ByteTensor(tuple(map(lambda s:s is not None, next_state)))
    non_final_next_states = Variable(FloatTensor([self.index_to_onehot(s) for s in next_state if s is not None]), volatile=True)

    state_batch = Variable(FloatTensor([self.index_to_onehot(s) for s in state]))
    action_batch = Variable(LongTensor([a for a in action]))
    reward_batch = Variable(FloatTensor(reward))
#     print('state_batch:',state_batch)
#     print('self.Q(state_batch):',self.Q(state_batch))
#     print('action_batch:',action_batch)
    state_action_values = self.Q(state_batch).gather(1, action_batch.view(-1,1))
    
    next_state_values = Variable(torch.zeros(len(batch)).type(FloatTensor))
    next_state_values[non_final_mask] = self.Q(non_final_next_states).max(1)[0]
    
    expected_state_action_values = (next_state_values * self._gamma) + reward_batch
    expected_state_action_values = Variable(expected_state_action_values.view(-1,1).data)
    
#     print('state_action_values',state_action_values)
#     print('expected_state_action_values',expected_state_action_values)
    loss = self.loss_fn(state_action_values, expected_state_action_values)
#     print('loss', loss.data[0])
    self.optimizer.zero_grad()
    loss.backward()
    
    # gradient clipping
    for param in self.Q.parameters():
        param.grad.data.clamp_(-1, 1)

    self.optimizer.step()


  def show_policy(self):
    # greedy policy
# SFFF
# FHFH
# FFFH
# HFFG
    action_name_map = ['left','down','right','up','none']
    for i in range(4):
      policy = ""
      for j in range(4):
        if (i * 4 + j) in [5,7,11,12,15]:
          policy += "%2d: %-10s" %(state, 'None')
          continue
        state = i*4+j
        action_value = self.predict(state)
        action_index = action_value.index(max(action_value))
        policy += "%2d: %-10s" %(state, action_name_map[action_index])
      print(policy)

  def train(self, env, episode, epoch, batch_size = 32):
    for i in range(episode):
      s0 = env.reset()
      a0 = self.pick_action(s0)
      episode_ended = False
      while not episode_ended:
        (s1, reward, episode_ended, info) = env.step(a0)
        if reward > 0.0:
          print('reward:', reward)
          
        if not episode_ended:
          if reward <= 0.0:
            reward = -0.1
          a1 = self.pick_action(s1)
        else:
          if reward <= 0.0:
            reward = -1.0
          a1 = None
          s1 = None
        self.transition_history.add_transition(s0,a0,reward,s1)
        s0 = s1
        a0 = a1
      self.update_Q(self.transition_history.transitions)
      self.transition_history.clear()
#     iteration = epoch * len(self.transition_history.transitions) // batch_size
#     for i in range(iteration):
#       batch = self.transition_history.get_sample_transition(batch_size)
#       self.update_Q(batch)


In [50]:
env = gym.make('FrozenLakeNotSlippery4x4-v0')
agent = q_approx(16)

In [55]:
# print(agent.Q.state_dict())
# print('='*50)
agent.show_policy()
for i in range(100):
  agent.train(env,5000,1)
  if (i + 1) % 10 == 0:
    print('i:',i)
    agent.show_policy()
#     print(agent.Q.state_dict())
#     print('='*50)

 0: right      1: right      2: down       3: right     
 4: down       4: None       6: down       6: None      
 8: right      9: right     10: down      10: None      
10: None      13: right     14: right     14: None      
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
i: 9
 0: right      1: right      2: down       3: right     
 4: down       4: None       6: down       6: None      
 8: right      9: down      10: down      10: None      
10: None      13: right     14: right     14: None      
reward: 1.0
i: 19
 0: right      1: right      2: down       3: right     
 4: down       4: None       6: down       6: None      
 8: right      9: down      10: down      10: None      
10: None      13: right     14: right     14: None      
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
reward: 1.0
i: 29
 0: right      1: right      2: down       3: right     
 4: down       4: None       6: down       6: None      
 8: r

In [54]:
action_name_map = ['left','down','right','up','none']
for i in range(4):
  policy = ""
  for j in range(4):
    if (i * 4 + j) in [5,7,11,12,15]:
      continue
    state = i*4+j
    action_value = agent.predict(state)
    action_index = action_value.index(max(action_value))
    print('state:', state, action_name_map[action_index], action_value)

#     policy += "%2d: %-10s" %(state, action_name_map[action_index])
#   print(policy)

state: 0 right [-0.3841942250728607, -0.3387400805950165, -0.31577813625335693, -0.38410425186157227]
state: 1 right [-0.384112685918808, -0.9999314546585083, -0.23986807465553284, -0.31474339962005615]
state: 2 down [-0.31462711095809937, -0.15629145503044128, -0.16590292751789093, -0.2260712832212448]
state: 3 right [-0.2275475561618805, -0.2446267306804657, -0.0822184681892395, -0.08603277802467346]
state: 4 down [-0.33876967430114746, -0.2653079926967621, -0.9999646544456482, -0.38393115997314453]
state: 6 down [-0.7284090518951416, -0.06479938328266144, -0.2104904055595398, -0.11793595552444458]
state: 8 right [-0.2653837502002716, -0.999298632144928, -0.18375496566295624, -0.3385081887245178]
state: 9 right [-0.265155553817749, -0.09389536827802658, -0.09310631453990936, -0.969906210899353]
state: 10 down [-0.1804787516593933, 0.005862134508788586, -0.39025938510894775, -0.12580232322216034]
state: 13 right [-0.9697795510292053, -0.08061707019805908, 0.005955095868557692, -0.1414